# Description
Create SWBD data for Montreal Forced Aligner

In [1]:
import os
from glob import iglob
from tqdm import tqdm
import shutil
import re 

In [2]:
PATH_AUDIO = '../Data/SWBD/swb1_LDC97S62'
PATH_TEXT = '../Data/SWBD/swb1_dialogact_annot'
PATH_MFA_SWBD = '../Data/SWBD/MFA_formatted'

In [3]:
auds = dict()
for f1 in iglob(os.path.join(PATH_AUDIO, "**"), recursive=True): 
    if f1.endswith('.wav'):
        f1_name = os.path.basename(f1)
        auds[f1_name] = f1

In [4]:
print(auds['sw04771.wav'])

../Data/SWBD/swb1_LDC97S62/swb1_d4/data/sw04771.wav


In [5]:
for f in tqdm(iglob(os.path.join(PATH_TEXT, '**'), recursive=True)):
    if f.endswith('.utt'):
        to_match = "sw0{}.wav".format(f[-8:-4])
        try:
            auds[to_match]
            shutil.copy(auds[to_match], PATH_MFA_SWBD)
        except KeyError:
            continue

1175it [00:09, 124.57it/s]


In [30]:
utt = re.compile("utt*.:")
double_brackets = re.compile("<<.*>>")
brackets = re.compile("<.*>")
square = re.compile("[[.*]]")

#upper_single = re.compile(r'^[A-Z\d]+$')
for f in tqdm(iglob(os.path.join(PATH_MFA_SWBD, '**'), recursive=True)):
    new_file = ""
    if f.endswith('.utt'):
        with open(f) as utts: 
            for line in utts.readlines():
                if re.search(utt, line): 
                    idx = re.search(utt, line).span()[1]
                    line = line[idx:]
                    if re.search(' /', line): 
                        end = re.search(' /', line).span()[0]
                        line = line[:end]   
                    line = double_brackets.sub('', line)
                    line = brackets.sub('', line)
                    line = square.sub('', line)
                    line = line.replace('{', '')\
                              .replace('}', '')\
                              .replace('[', '')\
                              .replace(']','')\
                              .replace('+', '')\
                              .replace('-', '')\
                              .replace('--', '')\
                              .replace('((', '')\
                              .replace('))', '')\
                              .replace('#', ' ')\
                              .replace(' F ', ' ')\
                              .replace(' C ', ' ')\
                              .replace(' E ', ' ')\
                              .replace(' D ', ' ')\
                              .replace(' / ', ' ')\
                              .replace(' .', '.')\
                              .replace(' ,', ',')
                    new_file += (line)
        new_file = ' '.join(new_file.split())
        with open("{}/sw0{}.lab".format(PATH_MFA_SWBD, os.path.basename(f)[-8:-4]), "w+") as fout: 
            fout.write(new_file)
         

2311it [00:02, 889.47it/s]
